In [12]:
# packages
import os
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from imblearn.over_sampling import SMOTE

from xgboost import XGBClassifier

from sklearn.metrics import classification_report, recall_score, confusion_matrix

from matplotlib import pyplot as plt

In [2]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
df = pd.read_csv('/content/drive/My Drive/ms_wind_curtailment_prediction/lagged_curtailment_target_features.csv', sep = ';', index_col=0)

In [5]:
# preprocessing pipelines
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

smote = SMOTE(random_state=13)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_train_preprocessed, y_train_preprocessed = smote.fit_resample(X_train_preprocessed, y_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [ ]:
# create model instance
xgboost_class = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic', random_state = 9)

In [17]:
# define X and y
X = df.drop(['redispatch', 'level'], axis = 1)
y = df['redispatch']

from sklearn.metrics import recall_score, confusion_matrix

# Initialize lists to store evaluation metrics
recall_scores = []
conf_matrices = []

# Perform forward chaining cross-validation
for train_index, test_index in tscv.split(X):
    X_train_preprocessed, X_test_preprocessed = X.iloc[train_index], X.iloc[test_index]
    y_train_preprocessed, y_test_preprocessed = y.iloc[train_index], y.iloc[test_index]

    xgboost_class.fit(X_train_preprocessed, y_train_preprocessed)

    y_pred = xgboost_class.predict(X_test_preprocessed)

    y_pred_binary = [1 if pred > 0.4 else 0 for pred in y_pred]

    # Calculate recall
    recall = recall_score(y_test_preprocessed, y_pred_binary)
    recall_scores.append(recall)

    # Calculate confusion matrix
    conf_matrix = confusion_matrix(y_test_preprocessed, y_pred_binary)
    conf_matrices.append(conf_matrix)

# Print average recall across all folds
print("Average Recall Score:", sum(recall_scores) / len(recall_scores))

# Print average confusion matrix across all folds
print("Average Confusion Matrix:")
print(sum(conf_matrices) / len(conf_matrices))

Average Recall Score: 0.0026578247698612395
Average Confusion Matrix:
[[2.1606e+04 2.9600e+01]
 [1.6684e+03 5.0000e+00]]


In [14]:
# Split the entire dataset into training and test sets
cutoff_time = "2023-12-29 23:45:00"

# train data
train = df[df.index < cutoff_time]
X_train = train.drop(['redispatch', 'level'], axis = 1)
y_train = train['redispatch']

# test data
test = df[df.index >= cutoff_time]
X_test = test.drop(['redispatch', 'level'], axis = 1)
y_test = test['redispatch']

# weights
# weights = df['level'] ## to be done

In [6]:
# create model instance
xgboost_class = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic', random_state = 9)
# fit model
xgboost_class.fit(X_train_preprocessed, y_train_preprocessed)
# make predictions
y_pred = xgboost_class.predict(X_test_preprocessed)

In [7]:
# evaluate
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.85      0.90     32458
         1.0       0.15      0.35      0.21      2484

    accuracy                           0.82     34942
   macro avg       0.55      0.60      0.55     34942
weighted avg       0.89      0.82      0.85     34942

